# Classification of Lab Samples

In [1]:
import init
from common import constants as cn
from common.trinary_data import TrinaryData
from common.data_provider import DataProvider
from common_python.plots import util_plots
from common_python.classifier import classifier_ensemble
from common_python.classifier import classifier_collection
from common import transform_data

import collections
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import cross_val_score

%matplotlib inline

In [2]:
data = TrinaryData()
data.df_X.head()

,0,1,2,3,4,5,6,7,8,9,...,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514
T2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,1,1,1,1,1,1,1,1
T3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,1,1,1,1,1,1,1,1
T4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,1,1,1,1,1,1,1,1
T5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,1,1,1,1,1,1,1,1
T6,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,1,1,1,1,1,1,1,1


In [3]:
df_sampleAM = transform_data.trinaryReadsDF(
    csv_file="AM_MDM_Mtb_transcripts_DEseq.csv")
df_sampleAW = transform_data.trinaryReadsDF(
"AW_plus_v_AW_neg_Mtb_transcripts_DEseq.csv")

normalizeReadsDF: could not find gene in df_gene_description: AS1726
normalizeReadsDF: could not find gene in df_gene_description: AS1890
normalizeReadsDF: could not find gene in df_gene_description: ASdes
normalizeReadsDF: could not find gene in df_gene_description: ASpks
normalizeReadsDF: could not find gene in df_gene_description: MT0009
normalizeReadsDF: could not find gene in df_gene_description: MT0012
normalizeReadsDF: could not find gene in df_gene_description: MT0025
normalizeReadsDF: could not find gene in df_gene_description: MT0031
normalizeReadsDF: could not find gene in df_gene_description: MT0032
normalizeReadsDF: could not find gene in df_gene_description: MT0036
normalizeReadsDF: could not find gene in df_gene_description: MT0066.2
normalizeReadsDF: could not find gene in df_gene_description: MT0069
normalizeReadsDF: could not find gene in df_gene_description: MT0099.1
normalizeReadsDF: could not find gene in df_gene_description: MT0116.1
normalizeReadsDF: could not fi

normalizeReadsDF: could not find gene in df_gene_description: Rv1962A
normalizeReadsDF: could not find gene in df_gene_description: Rv1982A
normalizeReadsDF: could not find gene in df_gene_description: Rv1991A
normalizeReadsDF: could not find gene in df_gene_description: Rv2063A
normalizeReadsDF: could not find gene in df_gene_description: Rv2142A
normalizeReadsDF: could not find gene in df_gene_description: Rv2231A
normalizeReadsDF: could not find gene in df_gene_description: Rv2231B
normalizeReadsDF: could not find gene in df_gene_description: Rv2237A
normalizeReadsDF: could not find gene in df_gene_description: Rv2274A
normalizeReadsDF: could not find gene in df_gene_description: Rv2395A
normalizeReadsDF: could not find gene in df_gene_description: Rv2395B
normalizeReadsDF: could not find gene in df_gene_description: Rv2427A
normalizeReadsDF: could not find gene in df_gene_description: Rv2468A
normalizeReadsDF: could not find gene in df_gene_description: Rv2548A
normalizeReadsDF: co

normalizeReadsDF: could not find gene in df_gene_description: Rv0616A
normalizeReadsDF: could not find gene in df_gene_description: Rv0691A
normalizeReadsDF: could not find gene in df_gene_description: Rv0959A
normalizeReadsDF: could not find gene in df_gene_description: Rv1366A
normalizeReadsDF: could not find gene in df_gene_description: Rv1962A
normalizeReadsDF: could not find gene in df_gene_description: Rv1982A
normalizeReadsDF: could not find gene in df_gene_description: Rv1991A
normalizeReadsDF: could not find gene in df_gene_description: Rv2063A
normalizeReadsDF: could not find gene in df_gene_description: Rv2142A
normalizeReadsDF: could not find gene in df_gene_description: Rv2231A
normalizeReadsDF: could not find gene in df_gene_description: Rv2231B
normalizeReadsDF: could not find gene in df_gene_description: Rv2237A
normalizeReadsDF: could not find gene in df_gene_description: Rv2274A
normalizeReadsDF: could not find gene in df_gene_description: Rv2395A
normalizeReadsDF: co

In [4]:
df_sampleAM = df_sampleAM.T
df_sampleAM.head()

,AS1726,AS1890,ASdes,ASpks,MT0009,MT0012,MT0025,MT0031,MT0032,MT0036,...,Rvnt36,Rvnt37,Rvnt38,Rvnt39,Rvnt40,Rvnt41,Rvnt42,Rvnt43,Rvnt44,Rvnt45
AM_D20_1,1,1,1,1,1,1,1,1,1,1,...,-1,1,-1,0,-1,-1,-1,-1,-1,-1
AM_D20_3,1,1,1,1,1,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AM_D20_4,1,1,1,1,1,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AM_D20_5,1,1,1,1,1,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
MDM_D20_1,1,1,1,1,1,1,1,1,1,1,...,-1,0,-1,-1,-1,-1,-1,-1,-1,-1


In [5]:
df_sampleAW = df_sampleAW.T
df_sampleAW.head()

,AS1726,AS1890,ASdes,ASpks,MT0009,MT0012,MT0025,MT0031,MT0032,MT0036,...,Rvnt36,Rvnt37,Rvnt38,Rvnt39,Rvnt40,Rvnt41,Rvnt42,Rvnt43,Rvnt44,Rvnt45
AW_plus_1,1,1,1,1,1,1,1,1,1,1,...,0,0,-1,0,-1,0,-1,0,0,-1
AW_neg_1,1,1,1,1,1,1,1,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
AW_plus_3,1,1,1,1,1,1,1,1,1,1,...,0,0,-1,0,-1,-1,-1,-1,-1,-1
AW_neg_3,1,1,1,1,1,1,1,1,1,1,...,0,-1,-1,-1,-1,-1,-1,-1,-1,-1
AW_plus_4,1,1,1,1,1,1,1,1,1,1,...,0,0,-1,-1,-1,-1,-1,-1,-1,1


## Classification Validations
Classify T2-T25 and see if result is same as original class


In [6]:
provider = DataProvider(is_normalize=False)
provider.do()

In [12]:
def getTimeSample(time_index):
    df0 = provider.dfs_data[0]
    num = len(provider.dfs_data)
    ser = pd.Series(np.repeat(0, len(df0.index)), index=df0.index)
    for idx in range(num):
        ser += provider.dfs_data[idx][time_index]
    df = pd.DataFrame(ser/num)
    df_result = transform_data.trinaryReadsDF(df_sample=df)
    return df_result.T

In [13]:
getTimeSample(2).head()

,Rv0001,Rv0002,Rv0003,Rv0004,Rv0005,Rv0006,Rv0007,Rv0008c,Rv0009,Rv0010c,...,Rvnt36,Rvnt37,Rvnt38,Rvnt39,Rvnt40,Rvnt41,Rvnt42,Rvnt43,Rvnt44,Rvnt45
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
provider.dfs_data[1].head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
Rv0001,702.025149,779.516150,1070.330107,521.248835,966.936447,484.195823,555.040510,831.962319,596.055777,436.412031,...,506.720680,1530.537317,4200.945280,907.661559,1659.302171,1416.808965,5445.610535,4040.124851,5459.701691,3478.791893
Rv0002,220.542687,269.453675,294.591012,153.028891,341.369245,149.461221,201.155532,327.676794,242.204515,179.118978,...,177.739408,256.130243,691.319398,396.455696,802.463710,660.245464,1097.233307,802.762651,1318.670928,1119.141608
Rv0003,148.669755,187.943639,190.481241,106.776727,205.851134,116.123453,120.865985,202.136128,239.309429,146.433324,...,170.142117,112.541873,314.458794,485.423051,983.433213,798.439168,482.854483,351.746580,675.442802,654.404557
Rv0004,64.976837,78.559085,65.477927,42.324591,71.765955,55.447652,42.010891,79.341913,83.944223,56.747405,...,64.590251,37.974005,134.469445,186.848768,264.589160,253.882070,231.471136,131.026697,285.204421,257.650020
Rv0005,2092.706817,2597.726971,3710.897521,1814.472108,3728.950511,1691.858454,2279.187143,3903.237362,1298.180611,1604.445036,...,607.371497,2610.424762,8464.092664,947.115256,1741.190571,1454.655816,6385.647779,5896.320893,10056.039420,9844.403232


In [10]:
svm_ensemble = classifier_ensemble.ClassifierEnsemble(
        classifier_ensemble.ClassifierDescriptorSVM(), filter_high_rank=15, size=30)
df_X = data.df_X.copy()
df_X.columns = data.features
svm_ensemble.fit(df_X, data.ser_y)

In [27]:

prediced_classesp = [svm_ensemble.predict(getTimeSample(n)) for n in [24, 25]]

In [30]:
dfff = pd.concat([dff, *prediced_classesp])
dfff.index = range(2, 26)
dfff

,0,1,2,3,4
2,0.000000,1.000000,0.000000,0.000000,0.0
3,0.000000,1.000000,0.000000,0.000000,0.0
4,0.000000,1.000000,0.000000,0.000000,0.0
5,0.000000,1.000000,0.000000,0.000000,0.0
6,0.000000,1.000000,0.000000,0.000000,0.0
7,0.000000,1.000000,0.000000,0.000000,0.0
8,0.000000,0.000000,1.000000,0.000000,0.0
9,0.000000,0.000000,1.000000,0.000000,0.0
10,0.000000,0.066667,0.166667,0.766667,0.0
11,0.000000,0.000000,1.000000,0.000000,0.0


In [26]:
dff = pd.concat(prediced_classes)
dff.index = [2 + v for v in range(len(prediced_classes))]
dff

,0,1,2,3,4
2,0.0,1.000000,0.000000,0.000000,0.0
3,0.0,1.000000,0.000000,0.000000,0.0
4,0.0,1.000000,0.000000,0.000000,0.0
5,0.0,1.000000,0.000000,0.000000,0.0
6,0.0,1.000000,0.000000,0.000000,0.0
7,0.0,1.000000,0.000000,0.000000,0.0
8,0.0,0.000000,1.000000,0.000000,0.0
9,0.0,0.000000,1.000000,0.000000,0.0
10,0.0,0.066667,0.166667,0.766667,0.0
11,0.0,0.000000,1.000000,0.000000,0.0


In [23]:
prediced_classes[0]

,0,1,2,3,4
0,0.0,1.0,0.0,0.0,0.0


In [43]:
data.ser_y
df_expected = pd.DataFrame(data.ser_y)
df_expected.columns = ["columns"]
df_expected["values"] = 1.0
dff_expected = df_expected.pivot(columns="columns", values="values")
dff_expected = dff_expected.fillna(0)
dff_expected.index = [int(v[1:]) for v in dff_expected.index]
dff_expected 
df_diff = (dff_expected - dfff)
df_diff = df_diff.applymap(lambda v: np.abs(v))
df_diff.sum().sum()/len(df_diff)

0.022222222222222216

In [45]:
# Check if the sample trinary values are the same as the computed ones.
df_features = pd.concat([getTimeSample(n) for n in range(0, 26)])
df_features

,Rv0001,Rv0002,Rv0003,Rv0004,Rv0005,Rv0006,Rv0007,Rv0008c,Rv0009,Rv0010c,...,Rvnt36,Rvnt37,Rvnt38,Rvnt39,Rvnt40,Rvnt41,Rvnt42,Rvnt43,Rvnt44,Rvnt45
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-1,0,0,0,-1,0,-1
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-1,0,-1
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,-1,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-1,0,0,0,-1,0,-1
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


observations
1. Compare makeNormalized with provider.df_normalized
1. Deviation for T10 indicates an issue with the classifer

## Classification of Lab Samples

In [ ]:
svm_ensemble = classifier_ensemble.ClassifierEnsemble(
        classifier_ensemble.ClassifierDescriptorSVM(), filter_high_rank=15, size=30)
df_X = data.df_X.copy()
df_X.columns = data.features
svm_ensemble.fit(df_X, data.ser_y)

In [ ]:
svm_ensemble.predict(df_sampleAM)

In [ ]:
svm_ensemble.predict(df_sampleAW)